In [ ]:
import os
import pathlib


import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns

pd.options.plotting.backend = "plotly"

import warnings
warnings.filterwarnings("ignore")

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

Setting the date to April 1st 2022

In [ ]:
current_date = pd.to_datetime("2022-04-01")
print(color.BOLD +  f"Current Date := {current_date}" + color.END)
print(color.BOLD +  f"===================================" + color.END)

Current Date := 2022-04-01 00:00:00


Loading the pricing data from April

In [ ]:
bond_pricing = pd.read_parquet(
    "20220401-corporate_and_sovereign_bonds_pricing_data.parquet"
)

Filtering for good liquidity

In [ ]:
bond_pricing = bond_pricing.dropna(subset=["Liquidity_Score"])
bond_pricing = bond_pricing[bond_pricing["Liquidity_Score"] != 5.0]

Ensuring to only consider bonds with matzrity after the set data to only have active bonds

And making the maturity column a day to maturity column

In [ ]:
# note: out of bounds timestamp will be NaN
bond_pricing["Maturity"] = pd.to_datetime(bond_pricing["Maturity"], errors="coerce")
bond_pricing = bond_pricing.dropna(subset=["Maturity"])
bond_pricing = bond_pricing[bond_pricing["Maturity"] > current_date]

# maturity --> #day to maturity
bond_pricing["Maturity"] = (bond_pricing["Maturity"] - current_date).dt.days

Focusing on specific columns and deleting other columns

In [ ]:
# Meta columns (identifiers)
col_meta_names = ["ISIN", "CUSIP", "Ticker", "Short_name_of_Issuer", "Currency"]

# Numerical columns (pricing, yield, duration, and spreads)
col_num_names = [
    "Bid_Price", "Mid_Price", "Ask_Price", "Bid_YTM", "Mid_YTM", "Ask_YTM",
    "Bid_Ask_Price_Spread", "Dirty_Bid_Price", "Dirty_Mid_Price", "Dirty_Ask_Price",
    "Bid_Ask_Yield_Spread", "Bid_Macaulay_Duration", "Mid_Macaulay_Duration", "Bid_Modified_Duration",
    "Bid_Convexity", "Mid_Convexity", "Ask_Convexity", "Bid_Z_Spread", "Mid_Z_Spread", "Ask_Z_Spread",
    "Spread_vs_Benchmark_Bid", "Spread_vs_Benchmark_Mid", "Spread_vs_Benchmark_Ask"
]

# Categorical columns (bond features)
col_cat_names = ["Tier", "Coupon_Type", "Defaulted", "Perpetual", "isCallable"]

# Time-related columns (dates and maturity)
col_time_names = ["Maturity", "Liquidity_asof", "Trace_Last_Trade_Date"]

# Combine all columns into one list
col_names = [*col_meta_names, *col_num_names, *col_cat_names, *col_time_names]

# Filter the DataFrame to keep only the selected columns
bond_pricing = bond_pricing[col_names]

# Optional: Verify that the columns are correctly filtered
print(bond_pricing.columns)


Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Bid_Z_Spread', 'Mid_Z_Spread',
       'Ask_Z_Spread', 'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof',
       'Trace_Last_Trade_Date'],
      dtype='object')


Rows with NaNs

In [ ]:
# Count the number of rows with NaN values in any column
rows_with_nans = bond_pricing.isna().sum(axis=1) > 0

# Show how many rows have NaNs
num_rows_with_nans = rows_with_nans.sum()
print(f"Number of rows with NaN values: {num_rows_with_nans}")

# Show how many rows would remain after dropping rows with NaNs
num_rows_remaining = bond_pricing.shape[0] - num_rows_with_nans
print(f"Number of rows remaining after dropping NaNs: {num_rows_remaining}")

# Optional: You can also see the percentage of rows with NaNs
percent_rows_with_nans = (num_rows_with_nans.sum() / bond_pricing.shape[0]) * 100
print(f"Percentage of rows with NaN values: {percent_rows_with_nans:.2f}%")

Number of rows with NaN values: 78007
Number of rows remaining after dropping NaNs: 15737
Percentage of rows with NaN values: 83.21%


Checking which columns carry a lot of NaNs

In [ ]:
# Check the number of NaNs per column and sort them in descending order
nan_per_column = bond_pricing.isna().sum().sort_values(ascending=False)

# Display columns with NaNs, you can set a threshold to show only columns with significant NaNs (e.g., > 10%)
nan_per_column[nan_per_column > 0]


Trace_Last_Trade_Date      69323
Ask_Z_Spread               39588
Bid_Z_Spread               39584
Mid_Z_Spread               20174
Ask_Convexity              19536
Mid_Convexity              19536
Mid_Macaulay_Duration      19536
Bid_Macaulay_Duration      19528
Bid_Convexity              19528
Bid_Modified_Duration      19528
Spread_vs_Benchmark_Ask    11335
Spread_vs_Benchmark_Mid    11335
Spread_vs_Benchmark_Bid    11329
CUSIP                       7874
Bid_Ask_Yield_Spread        6753
Ask_YTM                     6751
Mid_YTM                     6747
Bid_YTM                     6743
Dirty_Mid_Price             2440
Dirty_Ask_Price             2440
Dirty_Bid_Price             2433
Tier                          75
Coupon_Type                   18
Bid_Ask_Price_Spread           7
Ask_Price                      7
Mid_Price                      7
dtype: int64

Removing columns with over 30% missing data

In [ ]:
# Define a threshold for NaNs (e.g., drop columns with more than 30% missing data)
threshold = 0.3  # 50% of missing data
columns_to_drop = nan_per_column[nan_per_column > len(bond_pricing) * threshold].index.tolist()

# Drop the columns with high NaNs
bond_pricing = bond_pricing.drop(columns=columns_to_drop)

# Check remaining columns after dropping
print(f"Columns removed: {columns_to_drop}")
print(f"Remaining columns: {bond_pricing.columns}")


Columns removed: ['Trace_Last_Trade_Date', 'Ask_Z_Spread', 'Bid_Z_Spread']
Remaining columns: Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof'],
      dtype='object')


Checking again the imapct of removing rows with NaN content

In [ ]:
# Count the number of rows with NaN values in any column
rows_with_nans = bond_pricing.isna().sum(axis=1) > 0

# Show how many rows have NaNs
num_rows_with_nans = rows_with_nans.sum()
print(f"Number of rows with NaN values: {num_rows_with_nans}")

# Show how many rows would remain after dropping rows with NaNs
num_rows_remaining = bond_pricing.shape[0] - num_rows_with_nans
print(f"Number of rows remaining after dropping NaNs: {num_rows_remaining}")

# Optional: You can also see the percentage of rows with NaNs
percent_rows_with_nans = (num_rows_with_nans.sum() / bond_pricing.shape[0]) * 100
print(f"Percentage of rows with NaN values: {percent_rows_with_nans:.2f}%")

Number of rows with NaN values: 30201
Number of rows remaining after dropping NaNs: 63543
Percentage of rows with NaN values: 32.22%


Removing the rows with NaNs

In [ ]:
# Drop rows with NaNs
bond_pricing = bond_pricing.dropna().reset_index(drop=True)

# Check how many rows are left after dropping NaNs
print(f"Number of rows after dropping NaNs: {len(bond_pricing)}")

# Check the first few rows to verify the changes
print(bond_pricing.head())

Number of rows after dropping NaNs: 63543
           ISIN      CUSIP  Ticker Short_name_of_Issuer Currency  Bid_Price   
0  NL0000003580  N4580ZKD8  NETHRS            Kdom Neth      EUR   86.91900  \
1  NL0000003531  N4580ZKF3  NETHRS            Kdom Neth      EUR   93.02500   
2  NL0000003572  N4580ZKH9  NETHRS            Kdom Neth      EUR   88.11700   
3  NL0000003523  N81728AH5  NETHRS            Kdom Neth      EUR   94.84992   
4  NL0000003564  N4580ZKJ5  NETHRS            Kdom Neth      EUR   89.23200   

   Mid_Price  Ask_Price  Bid_YTM  Mid_YTM  ...  Spread_vs_Benchmark_Bid   
0   86.99400   87.06900  1.02266  1.01634  ...                 48.46494  \
1   93.10000   93.17500  0.82681  0.81755  ...                 34.95224   
2   88.19200   88.26700  0.99472  0.98799  ...                 45.67049   
3   94.85492   94.85992  0.68186  0.68118  ...                 25.99862   
4   89.30700   89.38200  0.97178  0.96457  ...                 49.44915   

   Spread_vs_Benchmark_Mid  Spre

Viewing data

In [ ]:
bond_pricing

,ISIN,CUSIP,Ticker,Short_name_of_Issuer,Currency,Bid_Price,Mid_Price,Ask_Price,Bid_YTM,Mid_YTM,...,Spread_vs_Benchmark_Bid,Spread_vs_Benchmark_Mid,Spread_vs_Benchmark_Ask,Tier,Coupon_Type,Defaulted,Perpetual,isCallable,Maturity,Liquidity_asof
0,NL0000003580,N4580ZKD8,NETHRS,Kdom Neth,EUR,86.91900,86.99400,87.06900,1.02266,1.01634,...,48.46494,48.09788,47.73142,SNRFOR,Fixed,False,False,False,5037,1648684800000000000
1,NL0000003531,N4580ZKF3,NETHRS,Kdom Neth,EUR,93.02500,93.10000,93.17500,0.82681,0.81755,...,34.95224,34.55710,34.16279,SNRFOR,Fixed,False,False,False,3211,1648684800000000000
2,NL0000003572,N4580ZKH9,NETHRS,Kdom Neth,EUR,88.11700,88.19200,88.26700,0.99472,0.98799,...,45.67049,45.26352,44.85716,SNRFOR,Fixed,False,False,False,4672,1648684800000000000
3,NL0000003523,N81728AH5,NETHRS,Kdom Neth,EUR,94.84992,94.85492,94.85992,0.68186,0.68118,...,25.99862,26.24484,26.49107,SNRFOR,Fixed,False,False,False,2846,1648684800000000000
4,NL0000003564,N4580ZKJ5,NETHRS,Kdom Neth,EUR,89.23200,89.30700,89.38200,0.97178,0.96457,...,49.44915,49.25931,49.07012,SNRFOR,Fixed,False,False,False,4307,1648684800000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63538,DE000WGZ7VT6,D966BZEL8,DZBK,DZ Bk AG,EUR,102.07868,102.13562,102.19261,0.60013,0.57588,...,62.58774,61.08774,59.58774,SNRLAC,Fixed,False,False,False,857,1648684800000000000
63539,DE000WGZ8FT7,D966CKBL3,DZBK,DZ Bk AG,EUR,99.75000,99.93250,100.11500,1.27845,1.22084,...,115.62189,110.55531,105.50259,SNRLAC,Fixed,False,False,False,1193,1648684800000000000
63540,DE000WGZ8NG8,D966CKRG7,DZBK,DZ Bk AG,EUR,100.16022,100.23530,100.31046,0.85515,0.83428,...,64.94519,63.44519,61.94519,SNRLAC,Fixed,False,False,False,1342,1648684800000000000
63541,DE000WGZ8P49,D966CKWV8,DZBK,DZ Bk AG,EUR,100.48052,100.55795,100.63545,0.86977,0.84890,...,66.40733,64.90733,63.40733,SNRLAC,Fixed,False,False,False,1382,1648684800000000000


Loading the rating data

In [ ]:
bond_rating = pd.read_parquet("20220401-Instrument_Rating_reference_data.parquet")

bond_rating["instrumentRating.effectiveDate"] = pd.to_datetime(bond_rating["instrumentRating.effectiveDate"])
bond_rating = bond_rating.dropna(subset=['isin'])
bond_rating = bond_rating.drop_duplicates().reset_index(drop=True)
bond_rating = bond_rating[bond_rating['instrumentRating.provider'].isin(['SNP', 'MOODYS'])]
bond_rating = bond_rating.drop(columns=['instrumentRating.endDate','instrumentRating.outlook','instrumentRating.watch'])

In [ ]:
bond_rating

,id,isin,cusip,instrumentRating.directionOfChange,instrumentRating.effectiveDate,instrumentRating.provider,instrumentRating.rating,instrumentRating.type
0,9,JP356680BBC5,J05523CL7,Downgrade,2021-06-25,MOODYS,A2,Long-Term Debt Rating Senior Unsecured LC
1,1336200,US12489WMR24,12489WMR2,DECISION NOT TO RATE,2005-08-29,MOODYS,NR,Long-Term Debt Rating Senior Subordinate LC
3,1336201,US63937HAG39,63937HAG3,None,2016-11-17,SNP,NR,Long-Term Local Currency
4,1336202,US12667G5S86,12667G5S8,None,2018-02-06,SNP,NR,Long-Term Local Currency
6,1336204,US93935PAH29,93935PAH2,None,2018-10-16,MOODYS,WR,Long-Term Debt Rating Senior Secured LC
...,...,...,...,...,...,...,...,...
4101290,33240396,US271014LK61,271014LK6,None,2001-05-31,SNP,AAA,Long-Term Rating
4101291,33240395,US271014LH33,271014LH3,None,2001-05-31,MOODYS,Aaa,Long-Term Rating
4101292,33240395,US271014LH33,271014LH3,None,2001-05-31,SNP,AAA,Long-Term Rating
4101293,33240394,US271014LJ98,271014LJ9,None,2001-05-31,MOODYS,Aaa,Long-Term Rating


In [ ]:
test = bond_rating.copy()

In [ ]:
# Pivot the ratings
ratings_pivot = bond_rating.pivot_table(index='isin', columns='instrumentRating.provider', values='instrumentRating.rating', aggfunc=lambda x: ', '.join(str(v) if v is not None else '' for v in x))
ratings_pivot.columns = [f'{col}' for col in ratings_pivot.columns]  # Rename columns

# Pivot the direction of change
direction_pivot = bond_rating.pivot_table(index='isin', columns='instrumentRating.provider', values='instrumentRating.directionOfChange', aggfunc=lambda x: ', '.join(str(v) if v is not None else '' for v in x))
direction_pivot.columns = [f'instrumentRating.directionOfChange.{col}' for col in direction_pivot.columns]  # Rename columns

# Merge both pivot tables back with original data
bond_rating = bond_rating[['isin']].drop_duplicates().merge(ratings_pivot, on='isin', how='left').merge(direction_pivot, on='isin', how='left')

# Display result
display(bond_rating)


,isin,MOODYS,SNP,instrumentRating.directionOfChange.MOODYS,instrumentRating.directionOfChange.SNP
0,JP356680BBC5,A2,NaN,Downgrade,NaN
1,US12489WMR24,NR,NaN,DECISION NOT TO RATE,NaN
2,US63937HAG39,NaN,NR,NaN,
3,US12667G5S86,NaN,NR,NaN,
4,US93935PAH29,WR,NR,,
...,...,...,...,...,...
2019473,US59261AVD61,NaN,NR,NaN,Not Rated
2019474,US59261AUZ82,NaN,NR,NaN,Not Rated
2019475,US271014LK61,Aaa,AAA,,
2019476,US271014LH33,Aaa,AAA,,


How many times both SNP and MOODY have NaN values

In [ ]:
count_missing = ((bond_rating['MOODYS'].isna()) & (bond_rating['SNP'].isna())).sum()
print(count_missing)

0


Check the NaNs per column

In [ ]:
# Check the number of NaNs per provider
na_counts = bond_rating.isna().sum(axis=0)
print("NaN counts per column:")
print(na_counts)

NaN counts per column:
isin                                              0
MOODYS                                       885075
SNP                                          568178
instrumentRating.directionOfChange.MOODYS    885075
instrumentRating.directionOfChange.SNP       568178
dtype: int64


Drop rows with no ISIN and delete ISIN duplicates (for ISING rows with all columns the same)

In [ ]:
# # Only use rows where ISIN is not null
# bond_rating = bond_rating[bond_rating["isin"].notna()]

# # Drop duplicates and reset index
# bond_rating = bond_rating.drop_duplicates().reset_index(drop=True)

# # Print the first 3 rows to check the data
# print(bond_rating.head(3))

Number of rows in the dataset

In [ ]:
# Count the number of rows in the bond_rating dataset
print(f"Instrument rating number := {bond_rating.shape[0]}")

Instrument rating number := 2019478


Rating providers and rating values

In [ ]:
# Check unique providers and ratings
print("\nUnique ratings:")
print({col: bond_rating[col].unique().tolist() for col in ['MOODYS','SNP']})


Unique ratings:
{'MOODYS': ['A2', 'NR', nan, 'WR', 'Baa2, Baa2', 'Aa1', 'A1', 'WR, WR', 'Aaa', 'Baa3', 'Ca', 'Aa1, Aa1', 'Caa3', 'Baa1', 'Ba1', 'C', 'Ba2', 'Aa2', 'Aa3', 'Baa2', 'A3', 'A1, A1', 'Baa2, Ba1', 'Aa2, A1', 'Aa1, A3', 'Aa2, Aa2', 'Aaa, Aa3', 'Aaa, A1', 'Aa2, A2', 'A2, A1, A1', '85 - LGD5, B3', 'Aa3, Aa3', 'A2, WR', 'A1, A2', 'Aaa, Aa1', 'B1', 'Aa2, VMIG 1', 'Aa1, A2', 'B2', 'A1, Baa1', 'Aa2, Aa3', 'A1, Aa3', '45 - LGD3, Ba1', 'A1, A3', '71 - LGD5, B1', '81 - LGD5, B2', '84 - LGD5, Caa1', 'Aaa, VMIG 1', '73 - LGD5, Ba3', '30 - LGD3, Ba3', 'A3, Aa3', 'NR, NR', 'Baa1, Baa1', '40 - LGD3, B1', '77 - LGD5, Caa1', 'B3', 'WR, Baa3', '92 - LGD6, Caa1', '77 - LGD5, B3', '58 - LGD4, Ba3', 'Caa2', '93 - LGD6, Caa1', 'NR, WR', 'Aa1, Aa3', 'Aaa, Aaa', 'Aaa, Aa2', '50 - LGD4, Ba1', 'A2, Aa1', 'A3, A3', 'Baa2, WR', '53 - LGD4, B1', '51 - LGD4, B3', 'Caa1', 'Aaa, A2', 'WR, WR, WR', 'Aa1, Aa2', 'A1, Baa3', 'Aa1, Baa1', 'WR, WR, WR, WR', '61 - LGD4, B2', 'A2, A1, A2', 'WR, Caa2', '65 - LGD4, 

In [ ]:
bond_rating[bond_rating['isin']=='ZM1000005000']

,isin,MOODYS,SNP,instrumentRating.directionOfChange.MOODYS,instrumentRating.directionOfChange.SNP


Number of bonds per rating

In [ ]:
# Check how many times each rating appears
pd.options.display.max_rows = 100
print("\nRating counts:")
print(bond_rating["MOODYS"].value_counts(),'\n')
print(bond_rating["SNP"].value_counts())


Rating counts:
MOODYS
WR                       347082
Aa2                      116213
Aaa                      101313
NR                        83534
Aa3                       77854
                          ...  
Baa2, Ba3                     1
Aaa, Aa2, VMIG 1              1
A1, Aaa.mx                    1
Ba2, 50 - LGD4                1
WR, WR, 93 - LGD6, B1         1
Name: count, Length: 890, dtype: int64 

SNP
NR                  443232
NR, NR              172937
AA                  162197
AA+                 106337
AAA                  97175
                     ...  
BB+, 2(80%), BB+         1
BB, 2(75%), BB           1
brAA+, 5(15%)            1
BB (sf), BB (sf)         1
mxAA, BBB-               1
Name: count, Length: 684, dtype: int64


Deleting duplicates in ISINs with multiple ratings from the same rating provider

In [ ]:
# # Sort by ISIN, provider, and effective date (descending to get most recent first)
# bond_rating = bond_rating.sort_values(
#     ["isin", "instrumentRating.provider", "instrumentRating.effectiveDate"],
#     ascending=[True, True, False]  # Ascending by ISIN and provider, but descending by date
# )

# # Drop duplicates, keeping the first (most recent) for each ISIN and rating provider combination
# bond_rating = bond_rating.drop_duplicates(subset=["isin", "instrumentRating.provider"], keep="first")

# # Show the cleaned-up data
# bond_rating.head()

Filter for SNP as this provider covers most ISINs

### Merge the pricing April data and the SNP rating

In [ ]:
print(bond_pricing.columns)
# Renaming the isin column to be the same across datasets
bond_rating = bond_rating.rename(columns={"isin": "ISIN"})
# Merge the bond pricing data with the bond rating data on the 'ISIN' column
bond_pricing = bond_pricing.merge(bond_rating, on="ISIN", how="left")
print(bond_pricing.columns)

Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof'],
      dtype='object')
Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Con

In [ ]:
# # Number of rows after merging but before dropping NaNs
# rows_after_merge = bond_pricing.shape[0]

# # Drop rows where the rating is NaN
# bond_pricing = bond_pricing.dropna(subset=["instrumentRating.rating"])

# # Number of rows after dropping NaNs
# rows_after_drop = bond_pricing.shape[0]

# # Calculate the difference
# rows_dropped = rows_after_merge - rows_after_drop

# # Print the results
# print(f"Rows after merging: {rows_after_merge}")
# print(f"Rows after dropping rows with no rating: {rows_after_drop}")
# print(f"Rows dropped due to missing ratings: {rows_dropped}")
# print(f"Percentage of rows dropped: {rows_dropped / rows_after_merge * 100:.2f}%")

# # Optionally, check the value counts of the ratings column
# print(bond_pricing["instrumentRating.rating"].value_counts())

Include January pricing data now to calculate returns instead of prices

In [ ]:
print(color.BOLD + f"Current Date is set to := {current_date}")

Current Date is set to := 2022-04-01 00:00:00


In [ ]:
# previous_date = current_date - pd.Timedelta("1D")
previous_date = pd.to_datetime("2022-01-03")
# load data
previous_bond_pricing = pd.read_parquet("20220103-corporate_and_sovereign_bonds_pricing_data.parquet")

In [ ]:
# Only look at the ISIN and the mid price columns and drop the rows for which these column entries are empty
# Also rename the mid price column
previous_bond_pricing = previous_bond_pricing[
    ["ISIN", "Mid_Price"]
].dropna().rename(columns={"Mid_Price": "Previous_Mid_Price"})

Merge the previous mid price from the march with the big bond pricing dataset (from April) including the ratings

In [ ]:
# Merging
bond_pricing = bond_pricing.merge(
    previous_bond_pricing, on="ISIN", how="left"
)

Calculating the returns

In [ ]:
# First checking if the previous date is set correctly
print(color.BOLD + f"Previous Date is set to := {previous_date}")
print(color.BOLD + f"Current Date is set to := {current_date}")

Previous Date is set to := 2022-01-03 00:00:00
Current Date is set to := 2022-04-01 00:00:00


In [ ]:
# Calculating the return as follows: (today's (April) mid price - previous (March) mid price)/ previous (March) mid price
bond_pricing["Return %"] = 100*(
    bond_pricing["Mid_Price"] - bond_pricing["Previous_Mid_Price"]
) / bond_pricing["Previous_Mid_Price"]

print(bond_pricing.columns)


Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof', 'MOODYS',
       'SNP', 'instrumentRating.directionOfChange.MOODYS',
       'instrumentRating.directionOfChange.SNP', 'Previous_Mid_Price',
       'Return %'],
      dtype='object')


In [ ]:
# Dropping the previous mid price column as it is not needed anymore
# bond_pricing = bond_pricing.drop(columns=["Previous_Mid_Price"])

print(col_num_names)
# Add return column to the other numerical columns
col_num_names.append("Return %")  # Add "Return" to numerical columns list
print(col_num_names)

print(bond_pricing.columns)

['Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM', 'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price', 'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration', 'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity', 'Mid_Convexity', 'Ask_Convexity', 'Bid_Z_Spread', 'Mid_Z_Spread', 'Ask_Z_Spread', 'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid', 'Spread_vs_Benchmark_Ask']
['Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM', 'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price', 'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration', 'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity', 'Mid_Convexity', 'Ask_Convexity', 'Bid_Z_Spread', 'Mid_Z_Spread', 'Ask_Z_Spread', 'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid', 'Spread_vs_Benchmark_Ask', 'Return %']
Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_Y

In [ ]:
# Print final shape of the bond pricing data
print(bond_pricing.shape)

(63543, 39)


In [ ]:
bond_pricing

,ISIN,CUSIP,Ticker,Short_name_of_Issuer,Currency,Bid_Price,Mid_Price,Ask_Price,Bid_YTM,Mid_YTM,...,Perpetual,isCallable,Maturity,Liquidity_asof,MOODYS,SNP,instrumentRating.directionOfChange.MOODYS,instrumentRating.directionOfChange.SNP,Previous_Mid_Price,Return %
0,NL0000003580,N4580ZKD8,NETHRS,Kdom Neth,EUR,86.91900,86.99400,87.06900,1.02266,1.01634,...,False,False,5037,1648684800000000000,NaN,NaN,NaN,NaN,96.57600,-9.921720
1,NL0000003531,N4580ZKF3,NETHRS,Kdom Neth,EUR,93.02500,93.10000,93.17500,0.82681,0.81755,...,False,False,3211,1648684800000000000,NaN,NaN,NaN,NaN,99.97100,-6.872993
2,NL0000003572,N4580ZKH9,NETHRS,Kdom Neth,EUR,88.11700,88.19200,88.26700,0.99472,0.98799,...,False,False,4672,1648684800000000000,NaN,NaN,NaN,NaN,97.24300,-9.307611
3,NL0000003523,N81728AH5,NETHRS,Kdom Neth,EUR,94.84992,94.85492,94.85992,0.68186,0.68118,...,False,False,2846,1648684800000000000,NaN,NaN,NaN,NaN,100.66740,-5.773945
4,NL0000003564,N4580ZKJ5,NETHRS,Kdom Neth,EUR,89.23200,89.30700,89.38200,0.97178,0.96457,...,False,False,4307,1648684800000000000,NaN,NaN,NaN,NaN,97.79800,-8.682182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63538,DE000WGZ7VT6,D966BZEL8,DZBK,DZ Bk AG,EUR,102.07868,102.13562,102.19261,0.60013,0.57588,...,False,False,857,1648684800000000000,NaN,NaN,NaN,NaN,104.09941,-1.886456
63539,DE000WGZ8FT7,D966CKBL3,DZBK,DZ Bk AG,EUR,99.75000,99.93250,100.11500,1.27845,1.22084,...,False,False,1193,1648684800000000000,NaN,NaN,NaN,NaN,103.40500,-3.358155
63540,DE000WGZ8NG8,D966CKRG7,DZBK,DZ Bk AG,EUR,100.16022,100.23530,100.31046,0.85515,0.83428,...,False,False,1342,1648684800000000000,NaN,NaN,NaN,NaN,103.54481,-3.196210
63541,DE000WGZ8P49,D966CKWV8,DZBK,DZ Bk AG,EUR,100.48052,100.55795,100.63545,0.86977,0.84890,...,False,False,1382,1648684800000000000,NaN,NaN,NaN,NaN,104.01863,-3.326981


Saving the completely merged dataset(April data, return column, rating)

In [ ]:
bond_pricing.to_parquet("Bond_Pricing_With_Returns_And_Ratings.parquet")

Exploring the organisations

In [ ]:
bond_rating_org = pd.read_parquet(
    "20220401-Organisation_Rating.parquet"
)
print(bond_rating_org.columns)
print(bond_pricing.columns)

Index(['id', 'organizationId', 'ticker', 'legacyparentTicker',
       'legacyTopParentTicker', 'parentTicker', 'topParentTicker',
       'organizationRating.directionOfChange',
       'organizationRating.effectiveDate', 'organizationRating.endDate',
       'organizationRating.outlook', 'organizationRating.provider',
       'organizationRating.rating', 'organizationRating.type',
       'organizationRating.watch'],
      dtype='object')
Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
      

In [ ]:
# Number of tickers ine ach dataset
num_bond_tickers = bond_pricing["Ticker"].nunique()
num_org_tickers = bond_rating_org["ticker"].nunique()

print(f"Unique tickers in bond dataset: {num_bond_tickers}")
print(f"Unique tickers in organization dataset: {num_org_tickers}")

Unique tickers in bond dataset: 8665
Unique tickers in organization dataset: 366027


In [ ]:
# Find overlapping tickers
# Convert tickers to sets (drop NaNs to avoid issues)
bond_tickers = set(bond_pricing["Ticker"].dropna().unique())
org_tickers = set(bond_rating_org["ticker"].dropna().unique())

# Find common tickers
common_tickers = bond_tickers.intersection(org_tickers)
num_common_tickers = len(common_tickers)

print(f"Tickers in both datasets: {num_common_tickers}")

coverage_percentage = (num_common_tickers / num_bond_tickers) * 100
print(f"Percentage of bond tickers covered by organization ratings: {coverage_percentage:.2f}%")


Tickers in both datasets: 8665
Percentage of bond tickers covered by organization ratings: 100.00%


This implies that all the tickers in the bond_pricing data are covered in the org ratings

Integrating the org ticker data into the bond_pricing data

In [ ]:
# Step 1: Keep only the latest organization rating for each ticker
bond_rating_org = bond_rating_org.sort_values(["ticker", "organizationRating.effectiveDate"], ascending=False)
bond_rating_org = bond_rating_org.drop_duplicates(subset=["ticker"], keep="first")

# Step 2: Merge with bond_pricing using "Ticker"
bond_pricing = bond_pricing.merge(
    bond_rating_org[["ticker", "organizationRating.provider", "organizationRating.rating",
                     "organizationRating.effectiveDate", "organizationRating.outlook"]],
    left_on="Ticker",
    right_on="ticker",
    how="left"
)

# Step 3: Drop the duplicate ticker column from organization ratings
bond_pricing = bond_pricing.drop(columns=["ticker"])

# Step 4: Rename columns for clarity
bond_pricing = bond_pricing.rename(columns={
    "organizationRating.provider": "Org_Rating_Provider",
    "organizationRating.rating": "Org_Rating",
    "organizationRating.effectiveDate": "Org_Rating_EffectiveDate",
    "organizationRating.outlook": "Org_Outlook"
})

# Step 5: Check the merge results
print(bond_pricing[["Ticker", "Org_Rating_Provider", "Org_Rating"]].head())


   Ticker Org_Rating_Provider Org_Rating
0  NETHRS               FITCH        AAA
1  NETHRS               FITCH        AAA
2  NETHRS               FITCH        AAA
3  NETHRS               FITCH        AAA
4  NETHRS               FITCH        AAA


Look at org rating providers and bond rating providers

In [ ]:
print(bond_pricing.head())

           ISIN      CUSIP  Ticker Short_name_of_Issuer Currency  Bid_Price   
0  NL0000003580  N4580ZKD8  NETHRS            Kdom Neth      EUR   86.91900  \
1  NL0000003531  N4580ZKF3  NETHRS            Kdom Neth      EUR   93.02500   
2  NL0000003572  N4580ZKH9  NETHRS            Kdom Neth      EUR   88.11700   
3  NL0000003523  N81728AH5  NETHRS            Kdom Neth      EUR   94.84992   
4  NL0000003564  N4580ZKJ5  NETHRS            Kdom Neth      EUR   89.23200   

   Mid_Price  Ask_Price  Bid_YTM  Mid_YTM  ...  MOODYS  SNP   
0   86.99400   87.06900  1.02266  1.01634  ...     NaN  NaN  \
1   93.10000   93.17500  0.82681  0.81755  ...     NaN  NaN   
2   88.19200   88.26700  0.99472  0.98799  ...     NaN  NaN   
3   94.85492   94.85992  0.68186  0.68118  ...     NaN  NaN   
4   89.30700   89.38200  0.97178  0.96457  ...     NaN  NaN   

   instrumentRating.directionOfChange.MOODYS   
0                                        NaN  \
1                                        NaN   
2 

In [ ]:
print(bond_pricing.columns)

Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof', 'MOODYS',
       'SNP', 'instrumentRating.directionOfChange.MOODYS',
       'instrumentRating.directionOfChange.SNP', 'Previous_Mid_Price',
       'Return %', 'Org_Rating_Provider', 'Org_Rating',
       'Org_Rating_EffectiveDate', 'Org_Outlook'],
      dtype='object')


In [ ]:
# List of columns you want to print
columns_to_print = [
    'ISIN',
    'Ticker',
    'Org_Rating_Provider',
    'Org_Rating',
    'MOODYS',
    'instrumentRating.directionOfChange.MOODYS',
    'SNP',
    'instrumentRating.directionOfChange.SNP'
]

# Print the selected columns
print(bond_pricing[columns_to_print].tail())  # Display the first few rows of these columns


               ISIN Ticker Org_Rating_Provider Org_Rating MOODYS   
63538  DE000WGZ7VT6   DZBK              MOODYS         a3    NaN  \
63539  DE000WGZ8FT7   DZBK              MOODYS         a3    NaN   
63540  DE000WGZ8NG8   DZBK              MOODYS         a3    NaN   
63541  DE000WGZ8P49   DZBK              MOODYS         a3    NaN   
63542  DE000WGZ8FS9   DZBK              MOODYS         a3    NaN   

      instrumentRating.directionOfChange.MOODYS  SNP   
63538                                       NaN  NaN  \
63539                                       NaN  NaN   
63540                                       NaN  NaN   
63541                                       NaN  NaN   
63542                                       NaN  NaN   

      instrumentRating.directionOfChange.SNP  
63538                                    NaN  
63539                                    NaN  
63540                                    NaN  
63541                                    NaN  
63542              

Sometimes the ratings between the instrument dataset and the organisation rating dataset dont match:

In [ ]:
bond_pricing[(bond_pricing['Org_Rating_Provider']=='SNP')&(bond_pricing['ISIN']=='PEP01000C2Z1')][['ISIN','SNP','Org_Rating_Provider','Org_Rating']]

,ISIN,SNP,Org_Rating_Provider,Org_Rating


How many bonds have no rating from both MOODYS and SNP

In [ ]:
count_missing = ((bond_pricing['MOODYS'].isna()) & (bond_pricing['SNP'].isna())).sum()
print(count_missing)

24510


In [ ]:
# Filter rows with a specific ticker (for example 'KFW')
specific_ticker = 'KFW'
filtered_data = bond_pricing[bond_pricing['Ticker'] == specific_ticker]

# Specify the columns to display
columns_to_display = ['ISIN', 'Ticker', 'Org_Rating_Provider', 'Org_Rating', 'SNP', 'MOODYS', 'instrumentRating.directionOfChange.SNP','instrumentRating.directionOfChange.MOODYS']

# Show the filtered rows with the specific columns
filtered_data = filtered_data[columns_to_display]

# Print the filtered data
print(filtered_data)


               ISIN Ticker Org_Rating_Provider Org_Rating  SNP MOODYS   
9      AU0000002073    KFW               FITCH         WD  AAA    Aaa  \
5970   CH0029008809    KFW               FITCH         WD  AAA    Aaa   
10217  XS1716607269    KFW               FITCH         WD  AAA    Aaa   
11816  XS1950905486    KFW               FITCH         WD  AAA    Aaa   
12054  XS1814900806    KFW               FITCH         WD  AAA    Aaa   
...             ...    ...                 ...        ...  ...    ...   
60588  AU000KFWHAF2    KFW               FITCH         WD  AAA    Aaa   
60589  AU000KFWHAA3    KFW               FITCH         WD  AAA    Aaa   
60590  AU000KFWHAE5    KFW               FITCH         WD  AAA    Aaa   
61345  DE000A2YNZ16    KFW               FITCH         WD  AAA    Aaa   
61507  CAD6426YAA24    KFW               FITCH         WD  AAA    Aaa   

      instrumentRating.directionOfChange.SNP   
9                                 New Rating  \
5970                       

## How does this make sense? Trying to understand the meaning behind the organisation provider and the instrument provider and their ratings

Saving the merged dataset

In [ ]:
bond_pricing.to_parquet("Bond_Pricing_With_Returns_Ratings_And_Org.parquet")
print(bond_pricing.columns)
print(bond_pricing['Currency'].unique())

Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Tier', 'Coupon_Type',
       'Defaulted', 'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof',
       'MOODYS', 'SNP', 'instrumentRating.directionOfChange.MOODYS',
       'instrumentRating.directionOfChange.SNP', 'Previous_Mid_Price',
       'Return', 'Return %', 'Org_Rating_Provider', 'Org_Rating',
       'Org_Rating_EffectiveDate', 'Org_Outlook'],
      dtype='object')
['UGX' 'XOF' 'CLF' 'EUR' 'AUD' 'BWP' 'CZK' 'KES' 'ZMW' 'CNY' 'IDR' 'BDT'
 'JPY' 'USD' 'NOK' 'KZT' 'PLN' 'PEN' 'GBP' 'HKD' 'ILS' 'CHF' 'SGD' 'ZAR'
 'PKR' 'MYR' 'BRL' 'NZD' 'SEK' 'CAD' 'MXN' 'MXV' 'INR' 'PHP' 'DEM' 'TZS'
 'TWD' 'UAH' 'NGN' 'TRY' 'COP' 'GEL' 'VND' 'EGP' 'CLP' 'LKR' 'HUF' 'GBX'
 'DKK' 'ARS' 'CRC' 'ISK' 'KRW' 'MAD' 'LBP' 'RON' 'RSD' 'NLG' 